How to Set it up
1. you need to create a secrets.json file with following structure
    ```json
    {
        "amp_client_id": "",
        "amp_client_secret": "",
        "amp_user_id": ""
    }
    ```
2. You need the aws token download to access the db

In [4]:
import pandas as pd
from tabulate import tabulate
import requests
import json
import boto3
from boto3.dynamodb.conditions import Key
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
secretsFile = open('secrets.json')
secrets = json.load(secretsFile)
amp_gw_url = 'https://apicorp.rockfin.com/openamp'
req_headers = {
    "xAmpUserId": secrets['amp_user_id'],
    "client_id": secrets['amp_client_id'],
    "client_secret": secrets['amp_client_secret'],
    "vnd.rockfin.appid": "206637" 
}
maskSSN = False

def getLoanApplicationNumber(loanNumber: str):
    applicationEndpoint = f'{amp_gw_url}/Application/Jacket/{loanNumber}'
    resp = requests.get(applicationEndpoint, verify=False, headers=req_headers)
    if resp.status_code == 200:
        jsonResp = json.loads(resp.text)
        return jsonResp[0]['applicationNumber']
    else:
        print(resp.status_code);
        print('Unable to get application number')
        return None


def getClientsInLoanFromAMP(applicationNumber):
    clientsEndpoint = f'{amp_gw_url}/Proxy/Client?ApplicationNumber={applicationNumber}'
    resp = requests.get(clientsEndpoint, verify=False, headers=req_headers)
    if resp.status_code == 200:
        jsonResp = json.loads(resp.text)
        clients = jsonResp['Clients']['Client']
        print("\nClients In AMP")
        clientNames = []
        clientGCIDs = []
        clientStatuses = []
        clientSSNs = []
        for client in clients:
            name = f'{client["FirstName"]} {client["MiddleName"] if client["MiddleName"] else "" } {client["LastName"]}'
            clientNames.append(name)
            clientGCIDs.append(client['GCUID'])
            status = 'Primary' if client['BorrowerNumber'] == 1 else 'Not Primary'
            ssn = client['SSN'] if not maskSSN else '********'
            clientStatuses.append(status)
            clientSSNs.append(ssn)
        df = pd.DataFrame({
            "Client Name": clientNames,
            "GCID": clientGCIDs,
            "Status": clientStatuses,
            "SSN": clientSSNs
        })
        tabulateDataFrame(df)
    else:
        print('Unable to get clients for loan')

def getClientsInMC(loanNumber):
    TABLE_NAME = "mortgageclient-data-prod-206763"
    dynamodb = boto3.resource(
        'dynamodb', region_name="us-east-2", verify=False)
    table = dynamodb.Table(TABLE_NAME)
    dynamoResponse = table.query(
        KeyConditionExpression=Key('LoanIdentifier').eq(str(loanNumber)) & Key('SortKey').begins_with('PI#'))

    if dynamoResponse:
        clientNames = []
        clientGCIDs = []
        clientStatuses = []
        clientSSNs = []
        recordStatuses = []
        spouseOnLoanGCID = []
        items = dynamoResponse['Items']
        # print(items)
        print('\nClients In MC')
        for count, item in enumerate(items):
            name = f'{item["FirstName"]} {item["MiddleName"] if item.get("MiddleName") else ""} {item["LastName"]}'
            status = 'Primary' if item['IsPrimary'] == True else 'Not Primary'
            ssn = item['SSN'] if not maskSSN else '********'
            recordStatus = 'Deleted' if item['IsDeleted'] == True else ''
            spouseOnLoan = item['SpouseOnLoanGCID'] if 'SpouseOnLoanGCID' in item else ''
            if item['IsPrimary'] == True and count != 0:
                clientNames.insert(0, name)
                clientGCIDs.insert(0, item['GCID'])
                clientStatuses.insert(0, status)
                clientSSNs.insert(0, ssn)
                recordStatuses.insert(0, recordStatus)
                spouseOnLoanGCID.insert(0, spouseOnLoan)
                continue
            clientNames.append(name)
            clientGCIDs.append(item['GCID'])
            clientStatuses.append(status)
            clientSSNs.append(ssn)
            recordStatuses.append(recordStatus)
            spouseOnLoanGCID.append(spouseOnLoan)

        df = pd.DataFrame({
            "Client Name": clientNames,
            "GCID": clientGCIDs,
            "Status": clientStatuses,
            "Record Status": recordStatuses,
            "SSN": clientSSNs,
            "SpouseGCID": spouseOnLoanGCID
        })
        tabulateDataFrame(df)


def getClients():
    loanIdentifier = int(input('Enter a Loan Number'))
    applicationNumber = getLoanApplicationNumber(loanIdentifier)
    if (applicationNumber == None):
        return
    print(f'Loan: {loanIdentifier}')
    print(f'Application Number: {applicationNumber}')
    getClientsInLoanFromAMP(applicationNumber)
    getClientsInMC(loanIdentifier)
       

def tabulateDataFrame(df):
    print(tabulate(df, headers = 'keys', tablefmt="psql"))

getClients()


Loan: 3503440453
Application Number: 49596933

Clients In AMP
+----+-----------------+-----------+-------------+-----------+
|    | Client Name     |      GCID | Status      |       SSN |
|----+-----------------+-----------+-------------+-----------|
|  0 | Michelle  Bova  | 152771515 | Primary     | 129766453 |
|  1 | Phil  Benedetto | 180287943 | Not Primary | 108705253 |
+----+-----------------+-----------+-------------+-----------+


ResourceNotFoundException: An error occurred (ResourceNotFoundException) when calling the Query operation: Requested resource not found